[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/sk-classroom/asc-qbit-nn/blob/main/exercise/exercise_01.ipynb)

# Learning Logistic Regression with a 1-bit Neural Network 


In this exercise, we will learn how to train a 1-bit neural network to perform logistic regression using the bitnet library. 


# Preparation

We will use the [bitnet library](https://github.com/kyegomez/BitNet), together with the standard Python stack.  


In [1]:
#! pip install bitnet
#! pip install transformers

In [3]:
from bitnet import *
import torch
from torch import optim
import torch.nn as nn

# Training 1-bit Neural Network 


## Task 

We will train a neural network to learn a logistic gate: 

| X1 | X2 | Y |
|----|----|---|
| 0  | 0  | 0 |
| 0  | 1  | 0 |
| 1  | 0  | 0 |
| 1  | 1  | 1 |

where X1 and X2 are the inputs, and Y is the output.

In [4]:
input = torch.tensor([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
output = torch.tensor(
    [
        [0.0],
        [0.0],
        [0.0],
        [1.0],
    ]
)

## Train a full-precision neural network

Let us first verify that a full-precision neural network can learn a logistic gate.

Our neural network should be composed of the following components: 
- We use 2 linear layers with ReLU activation, with 32 hidden neurons. 
- The output of the last layer should be passed through a sigmoid function. 



In [5]:
# TODO: Define the model
# Two linear layers with ReLU activation, followed by a sigmoid function.
model = nn.Sequential(
    nn.Linear(2, 32),
    nn.ReLU(),
    nn.Linear(32, 1),
    nn.Sigmoid(),
)

Train the model using `torch.nn.BCELoss`

In [8]:
from tqdm import tqdm

# TODO Define the optimizer
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# TODO Define the loss
loss_fn = nn.BCELoss()

# TODO Train the model
n_iter = 1000
pbar = tqdm(range(n_iter))

for i in pbar:
    optimizer.zero_grad()
    y = model.forward(input)
    loss = loss_fn(y, output)
    loss.backward()
    optimizer.step()
    pbar.set_postfix({"loss": loss.item()})

100%|██████████| 1000/1000 [00:00<00:00, 1039.93it/s, loss=0.000166]


In [53]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

y = model(input)
y = y.detach().numpy().reshape(-1)

pd.DataFrame({"Target": output.numpy().reshape(-1), "Predicted": y})

,Target,Predicted
0,0.0,1.050465e-07
1,0.0,2.166855e-04
2,0.0,1.350192e-04
3,1.0,9.996909e-01


# Train a 1-bit neural network

`bitnet` provides a `BitLinear` layer that can be used to train a 1-bit neural network. `BitLinear` can be used to replace `nn.Linear`. 

Let us now train a 1-bit neural network to learn a logistic gate. 

In [51]:
import bitnet

# TODO: Define the model
# Two linear layers with ReLU activation, followed by a sigmoid function.
model_1bit = nn.Sequential(
    bitnet.BitLinear(2, 32),
    nn.ReLU(),
    bitnet.BitLinear(32, 1),
    nn.Sigmoid(),
)

In [52]:
# TODO: Train the model
from tqdm import tqdm

# TODO Define the optimizer
optimizer = optim.AdamW(model_1bit.parameters(), lr=0.001)

# TODO Define the loss
loss_fn = nn.BCELoss()

# TODO Train the model
n_iter = 1000
pbar = tqdm(range(n_iter))

for i in pbar:
    optimizer.zero_grad()
    y = model_1bit.forward(input)
    loss = loss_fn(y, output)
    loss.backward()
    optimizer.step()
    pbar.set_postfix({"loss": loss.item()})

100%|██████████| 1000/1000 [00:01<00:00, 909.73it/s, loss=0.632]


In [54]:
y = model_1bit(input)
y = y.detach().numpy().reshape(-1)

pd.DataFrame({"Target": output.numpy().reshape(-1), "Predicted": y})

,Target,Predicted
0,0.0,0.500000
1,0.0,0.448024
2,0.0,0.467516
3,1.0,0.543533


# Why did 1-bit neural network fail to learn the logistic gate 🤔? 